In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/weather_data/daily/dublin 2024-01-01 to 2024-01-31.csv')

In [3]:
data.columns

Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

In [4]:
data['datetime'] = pd.to_datetime(data['datetime'])

In [5]:
data['datetime'] = data['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
print(data[['datetime']].tail())

               datetime
26  2024-01-27 00:00:00
27  2024-01-28 00:00:00
28  2024-01-29 00:00:00
29  2024-01-30 00:00:00
30  2024-01-31 00:00:00


In [6]:
data['sunrise'] = pd.to_datetime(data['sunrise'])
data['sunrise'] = data['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
print(data[['sunrise']].tail())

                sunrise
26  2024-01-27 08:18:23
27  2024-01-28 08:16:54
28  2024-01-29 08:15:22
29  2024-01-30 08:13:48
30  2024-01-31 08:12:12


In [7]:
data['sunset'] = pd.to_datetime(data['sunset'])
data['sunset'] = data['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')
print(data[['sunset']].tail())

                 sunset
26  2024-01-27 16:57:36
27  2024-01-28 16:59:30
28  2024-01-29 17:01:25
29  2024-01-30 17:03:21
30  2024-01-31 17:05:17


In [8]:
colums = [
    'datetime',
    'tempmax',
    'tempmin',
    'temp',
    'windspeed',
    'precip',
    'visibility',
    'humidity',
    'uvindex',
    'sunrise',
    'sunset'
]

In [9]:
df_sql = data[colums].copy()

In [10]:
rename = {
    'datetime' : 'time_stamp',
    'tempmax' : 'maxtemp_c',
    'tempmin' : 'mintemp_c',
    'temp' : 'avgtemp_c',
    'windspeed' : 'maxwind_kph',
    'precip' : 'totalprecip_mm',
    'visibility' : 'avgvis_km',
    'humidity' : 'avghumidity',
    'uvindex' : 'uv',
    'sunrise' : 'sunrise',
    'sunset' : 'sunset'
}

In [11]:
df_sql.rename(columns=rename, inplace=True)

In [12]:
df_sql.columns

Index(['time_stamp', 'maxtemp_c', 'mintemp_c', 'avgtemp_c', 'maxwind_kph',
       'totalprecip_mm', 'avgvis_km', 'avghumidity', 'uv', 'sunrise',
       'sunset'],
      dtype='object')

In [13]:
duplicates = df_sql.duplicated(keep=False)

In [14]:
print(df_sql[duplicates])

Empty DataFrame
Columns: [time_stamp, maxtemp_c, mintemp_c, avgtemp_c, maxwind_kph, totalprecip_mm, avgvis_km, avghumidity, uv, sunrise, sunset]
Index: []


In [15]:
df_sql = df_sql.drop_duplicates()

In [16]:
has_duplicates = duplicates.any()
print("Are there any duplicate rows?", has_duplicates)

Are there any duplicate rows? False


In [17]:
filename = f"dailyWeather.csv"
df_sql.to_csv(filename, index=False)

In [20]:
import pandas as pd
from sqlalchemy import create_engine

USER = 'musaddique333'
PASSWORD = '7483854963Mus#'
URL = 'comp3080.cho0moo0yp7i.us-east-1.rds.amazonaws.com'
PORT = '3306'
DB = 'dublinBikes29'

engine = create_engine(f'mysql+pymysql://{USER}:{PASSWORD}@{URL}:{PORT}/{DB}')

csv_file_path = 'dailyWeather.csv'
df = pd.read_csv(csv_file_path)

df['time_stamp'] = pd.to_datetime(df['time_stamp'])
df['sunrise'] = pd.to_datetime(df['sunrise'])
df['sunset'] = pd.to_datetime(df['sunset'])
df.to_sql('dailyWeather', con=engine, if_exists='append', index=False, method='multi')

31